# HuBMAP - Hacking the Kidney - Kaggle Competition

The [Kaggle competition page](https://www.kaggle.com/c/hubmap-kidney-segmentation)

Helpful Notebooks:
* [https://www.kaggle.com/markalavin/hubmap-tile-images-w-overlap-and-build-tfrecords](https://www.kaggle.com/markalavin/hubmap-tile-images-w-overlap-and-build-tfrecords)

## TODO:

* Look at impact of different affine matrices
* Look at impact of removing alpha channel on model size and performance
* Add Deepmind's architecture optimizer

In [1]:
TEST = False

## Setup

In [2]:
#! conda config --set always_yes True
#! conda install -c fastai -c pytorch fastai
#! conda update pytorch torchvision torchaudio fastai -c pytorch
#! conda update pytorch torchvision torchaudio cudatoolkit -c pytorch
#! conda install pandas
#! conda install -c conda-forge kaggle
#! conda install -c conda-forge tifffile
#! conda install -c conda-forge tqdm
# !conda install -c conda-forge matplotlib
#! conda install -c conda-forge pytorch-lightning

In [3]:
#! conda list

In [4]:
# I have no idea why the conda-forge version doesn't work

#!python -m pip install opencv-python

# If you are running this notebook on a server (like Linux on WSL2) you need the headless version of opencv
# The regular opencv requires GUI packages that serves dont have, and will raise an error
#!python -m pip install opencv-python-headless

# temporary solution to use tab complete - something wrong with jupyter jedi - need to downgrade
#!pip install jedi==0.17.2

# below 3 were needed for windows install on python3.9
#!pip install --upgrade --pre SimpleITK --find-links https://github.com/SimpleITK/SimpleITK/releases/tag/latest
#!python -m pip install scikit-build
#!python -m pip install --upgrade pip
#!pip install cmake
#!pip install torchio --upgrade

Ensure the finicky local CUDA is running

In [5]:
# First, import PyTorch
import torch
import torchvision
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F

# Check PyTorch version
torch.__version__
torch.cuda.is_available()

True

In [6]:
#from fastai.vision.all import *
#from fastai.imports import *
import pandas as pd
import numpy as np
import pytorch_lightning as pl
from pytorch_lightning.callbacks import Callback

# Need to put kaggle.json in /%USERS%/.kaggle folder (C:/Users/Craig/.kaggle)
#import kaggle

from pathlib import Path
import random

# Read tiff images
import tifffile
import cv2
from tqdm import tqdm
import torchio as tio

from PIL import Image
from IPython.display import display

import time

import matplotlib.pyplot as plt

# Memory management tools
import gc

In [7]:
path = Path()
#kaggle.api.competition_download_files("hubmap-kidney-segmentation", path=paLearner

Ensure you are about to download the data in the cvorrect directory

In [8]:
#path.ls()

Unzip the data in the correct folder - commented out so as to not repeat the unzipping

In [9]:
#import zipfile

#with zipfile.ZipFile(path/"hubmap-kidney-segmentation.zip", 'r') as zipref:
#    zipref.extractall(path)

## New EDA

- [x] Convert the encodings to masks
- [x] Show the kidney slices
- [] Tile the images and masks (can do seperately or when superimposed)
- [] Create an iterable of the tiled images
- [] Do EDA on the meta data

In [10]:
TEST_IMAGE_INDEX = 0

In [11]:
train_df = pd.read_csv(path/"train.csv").rename(columns={"id": "img_id"})

In [1]:
train_df.head()

NameError: name 'train_df' is not defined

In [12]:
path = Path()
#path.ls()

### Convert encodings to masks and show images

We have two options - to use the unencoded json data (in a separate file) or use the encoded data in the dataframe. We will use the latter

* **encodings**: represent the pixel number followed by how many pixels *over* (called *lengths* in our functions below)...??

## Test TIO

In [13]:
################
# Main Functions
################


def rle2mask(mask_rle, shape):
    '''
    mask_rle: encoding string value from csv
    shape: (width,height) of array to return
    Returns numpy array, 1 - mask, 0 - background
    '''
    s = mask_rle.split()
    # return a list of starting pixels and a list of lengths
    starts, lengths = [
        np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])
    ]
    # subtract 1 from every starting pixel
    starts -= 1
    ends = starts + lengths
    # calculate a background of 0 (empty) with size defined by image
    img = np.zeros(shape[0] * shape[1], dtype=np.uint8)
    # replace every 0 within each range with 1
    for lo, hi in zip(starts, ends):
        img[lo : hi] = 1
    return img.reshape(shape).T


##########
# Helpers
##########

def get_id_by_index(index, df=train_df):
    return df.iloc[index]['img_id']

def get_single_img(id, folder="train"):
    img = tifffile.imread(path/folder/(id+".tiff"))
    if len(img.shape) == 5:
        img = img.squeeze().transpose(1, 2, 0)
    return img

def show_single_img(id, **kwargs):
    return plt.imshow(get_single_img(id), **kwargs)

def show_img_by_index(index, df=train_df):
    return plt.imshow(tifffile.imread(path/"train"/(train_df.iloc[TEST_IMAGE_INDEX]['id']+".tiff")))

def get_single_encs(id, df=train_df):
    return df[df['img_id'] == id]['encoding'].array[0]

def get_mask(id, df=train_df, folder="train"):
    return rle2mask(
        get_single_encs(id, df=df),
        get_single_img(id, folder=folder).shape[::-1][1:]
    )

def show_single_img_and_mask(id):
    plt.figure(figsize=(16, 10))
    
    mask = get_mask(id)
    img = get_single_img(id)

    plt.subplot(1, 3, 1)
    plt.imshow(img)
    plt.title(f"Image", fontsize=18)
    
    plt.subplot(1, 3, 2)
    plt.imshow(img)
    plt.imshow(mask, cmap="hot", alpha=0.5)
    plt.title(f"Image + mask", fontsize=18)    
    
    plt.subplot(1, 3, 3)
    plt.imshow(mask, cmap="hot")
    plt.title(f"Mask", fontsize=18)    
    
    return plt.show()

def to_4d(img, input_chan_first=False, output_chan_first=True):
    if not len(img.shape)==3:
        raise ValueError("Function only converts 3D arrayto 4D array")
    return np.expand_dims(np.transpose(img, 
                   (0,1,2) if input_chan_first else (2,0,1)), 
                   3 if output_chan_first else 0)

def to_3d(img, input_chan_first=True, output_chan_first=False):
    if not len(img.shape)==4:
        raise ValueError("Function only converts 4D arrayto 3D array")
    return np.transpose(img.squeeze(), (0,1,2) if output_chan_first else (1,2,0))

def to_3chan(x, dim=1):
    return torch.cat((x,x,x), dim=dim)

def resizer(img, scale=5, show=False):
    """
    Returns an smaller array of the same dimensions, but converts to 3D to allow for resizing
    """
    scale_percent = scale # percent of original size
    im_dims = (len(img.shape) == 4)
    if im_dims:
        img = to_3d(img)
    width = int(img.shape[1] * scale_percent / 100)
    height = int(img.shape[0] * scale_percent / 100)
    dim = (width, height)
    img_reshaped = cv2.resize(img.numpy(), dim)
    if show:
        return plt.imshow(img_reshaped)
    if im_dims:
        return to_4d(img_reshaped)
    return img_reshaped

def squeeze_and_reshape(img_tensor, remove_alpha=False):
    if not isinstance(img_tensor, torch.Tensor):
        raise TypeError("Image needs to be a tensor")
    if len(img_tensor.shape) == 5:
        img_tensor = img_tensor.squeeze().permute(2, 1, 0)
    img_tensor = img_tensor.unsqueeze(2).permute(3,1,0,2)
    return img_tensor

def to_pil(image):
    # for 
    data = image.numpy().squeeze().T
    data = data.astype(np.uint8)
    image = Image.fromarray(data)
    w, h = image.size
    display(image)
    print() 

In [14]:
def remask(img, mask, tile, threshold=8, show=False):
    
    img_height = img.shape[1]
    img_width = img.shape[0]
    
    number_of_vertical_tiles = (img_height // tile)+1
    number_of_horizontal_tiles = (img_width // tile)+1
    
    #background = np.zeros((tile*number_of_horizontal_tiles, tile*number_of_vertical_tiles,3))[:img.shape[0],:img.shape[1],:img.shape[2]]
    
    tile_coords = []
    for h_idx in range(number_of_horizontal_tiles):
        for v_idx in range(number_of_vertical_tiles):
            tile_coords.append((h_idx+1, v_idx+1)) # +1 to remove 0 indexing

    cropped_images = []
    for h,v in tile_coords:
        cropped_images.append((h, v, img[tile*(h-1):tile*h, tile*(v-1):tile*v, :]))
        
    for horiz,vert,im in cropped_images:
        if not 0 in im.shape:      #required in case tile is 
            hsv = cv2.cvtColor(im, cv2.COLOR_BGR2HSV)
            h, s, v = cv2.split(hsv)
            if s.mean() < threshold:
                all_black = np.full((im.shape[0], im.shape[1]),2)
                mask[tile*(horiz-1):tile*horiz,tile*(vert-1):tile*vert] = all_black
                #im = im*0.
            #background[tile*(horiz-1):tile*horiz,tile*(vert-1):tile*vert,:] = im
    
    if show:
        plt.figure(figsize=(10, 10))

        plt.subplot(1, 3, 1)
        plt.imshow(img.astype('uint8'))
        plt.title(f"Image", fontsize=18)

        plt.subplot(1, 3, 2)
        plt.imshow(img.astype('uint8'))
        plt.imshow(mask.astype('uint8'), cmap="hot", alpha=0.5)
        plt.title(f"Image + mask", fontsize=18)    

        plt.subplot(1, 3, 3)
        plt.imshow(mask.astype('uint8'), cmap="hot")
        plt.title(f"Mask", fontsize=18)    

        plt.show()
    
    return mask


#img_id = get_id_by_index(7)
#img_id = '095bf7a1f'
#with tifffile.TiffFile(path/"train"/(img_id+".tiff")) as tif:
#    imgg = tif.asarray()
#print(imgg.shape)
#mask = get_mask(img_id)
#new_mask = remask(to_3d(squeeze_and_reshape(imgg)), mask, 1000)

In [15]:
def create_mask_df(df, directory):
    mask_list = []
    for idx,_ in tqdm(enumerate(df.iterrows()), total=len(df)):
        img_id = get_id_by_index(idx, df=df)
        with tifffile.TiffFile(path/directory/(img_id+".tiff")) as tif:
            base_im = tif.asarray()
            im_tensor = squeeze_and_reshape(torch.from_numpy(base_im)).numpy()
            mask = remask(to_3d(im_tensor), get_mask(img_id), 1000)
            mask_list.append((img_id, mask))
    return pd.DataFrame(mask_list, columns=["img_id", "mask"])

In [16]:
# Don't recreate the dataset everytime - pull from local directory if available as pickle file
# I understand pickles aren't safe - so only do this in your local env and never open an unfamiliar pickle file
if not (path/"new_masks.pkl").exists():
    new_masks = create_mask_df(train_df, "train")
    new_masks.to_pickle(path/"new_masks.pkl")
if (path/"new_masks.pkl").exists():
    new_masks = pd.read_pickle("new_masks.pkl")

In [17]:
def subject_creator(df):
    subjects_list = []
    for idx,_ in tqdm(enumerate(df.iterrows()), total=len(df)):
        
        affine = torch.tensor([[-1.,  0.,  0.,  0.],
       [ 0., -1.,  0.,  0.],
       [ 0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  1.]])
        
        img_id = get_id_by_index(idx, df=df)
        
        pic_list = [item for item in (path/"smaller/imgs").rglob("*") if not item.is_dir() and img_id in item.name]
        
        for pic in pic_list:
            pic_name = pic.name.split(".")[0]
            im = tio.ScalarImage(path/"smaller/imgs"/(pic_name+".tiff"))
            mask = tio.LabelMap(path/"smaller/masks"/(pic_name+"_mask.tiff"))

            subjects_list.append(tio.Subject(
                img = im,
                mask = mask,
                img_id = pic_name
            ))
    return subjects_list

In [18]:
def cut_image(img_id, mask_df):
    img = tio.Image(path/"train"/f"{img_id}.tiff").data
    if len(img.shape) != 4:
        raise ValueError("Tensor shape needs to have 4 dimensions")
    if img.shape[0] != 4:
        raise ValueError("First dimension must have 4 channels")
    vertical_tiles = img.shape[2] // 2
    horizontal_tiles = img.shape[1] // 2
    
    mask = torch.from_numpy(mask_df[mask_df["img_id"]==img_id]["mask"].values[0]).unsqueeze(0).unsqueeze(3)
    # I have managed to flip the axes somewhere and am too lazy or stubborn to fix the root issue. So need to permute axes
    mask = mask.permute(0,2,1,3)
    
    img1 = img[:,:horizontal_tiles,:vertical_tiles,:]
    mask1 = mask[:,:horizontal_tiles,:vertical_tiles,:]
    img2 = img[:,horizontal_tiles:,:vertical_tiles,:]
    mask2 = mask[:,horizontal_tiles:,:vertical_tiles,:]
    img3 = img[:,:horizontal_tiles,vertical_tiles:,:]
    mask3 = mask[:,:horizontal_tiles,vertical_tiles:,:]
    img4 = img[:,horizontal_tiles:,vertical_tiles:,:]
    mask4 = mask[:,horizontal_tiles:,vertical_tiles:,:]
    
    tio.Image(tensor=img1).save(path/"smaller/imgs"/f"{img_id}_1.tiff")
    tio.Image(tensor=mask1).save(path/"smaller/masks"/f"{img_id}_1_mask.tiff")
    tio.Image(tensor=img2).save(path/"smaller/imgs"/f"{img_id}_2.tiff")
    tio.Image(tensor=mask2).save(path/"smaller/masks"/f"{img_id}_2_mask.tiff")
    tio.Image(tensor=img3).save(path/"smaller/imgs"/f"{img_id}_3.tiff")
    tio.Image(tensor=mask3).save(path/"smaller/masks"/f"{img_id}_3_mask.tiff")
    tio.Image(tensor=img4).save(path/"smaller/imgs"/f"{img_id}_4.tiff")
    tio.Image(tensor=mask4).save(path/"smaller/masks"/f"{img_id}_4_mask.tiff")


#[cut_image(item, new_masks) for item in new_masks.img_id.tolist()]

### Custom Transform

In [19]:
#custom_normalization = tio.Lambda(lambda x: torch.half(x/255), types_to_apply=[tio.IMAGE])
custom_normalization = tio.Lambda(lambda x: (x/255).float(), types_to_apply=[tio.INTENSITY])

In [20]:
custom_reshape = tio.Lambda(lambda x: x[:3,...], types_to_apply=[tio.INTENSITY])

In [21]:
# unnecessary as I should find out why there are different shapes but I want to get to model building
def shuffle_axes(img_tensor):
    return img_tensor.permute(0,2,1,3)
reshuffle = tio.Lambda(shuffle_axes, types_to_apply=[tio.LABEL])

In [22]:
custom_shrink = tio.Lambda(lambda x: torch.tensor(resizer(x, 15)))

In [24]:
subjects_list = subject_creator(new_masks)
subjects_list_copy = subjects_list[:]     # needed because shuffle does in place
random.seed(57)
random.shuffle(subjects_list_copy)

train_subjects = subjects_list_copy[:round(len(subjects_list_copy)*0.8)]
valid_subjects = subjects_list_copy[round(len(subjects_list_copy)*0.8):]

#train_subjects = subject_creator(new_masks.iloc[:1])
#valid_subjects = subject_creator(new_masks.iloc[1:2])

100%|███████████████████████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 571.46it/s]


In [26]:
len(valid_subjects)

6

In [24]:
if TEST:
    test_subject = train_subjects[0]
    #to_pil(tio.RandomElasticDeformation()(test_subject["img"][tio.DATA]))
    
    small_test_image = custom_shrink(test_subject["img"][tio.DATA])
    
    max_displacement = 500, 400, 0
    #to_pil(tio.RandomElasticDeformation(max_displacement=max_displacement, num_control_points=10)(small_test_image))
    to_pil(tio.RandomElasticDeformation(max_displacement=max_displacement)(small_test_image))

In [25]:
# to test what data types there are in the dataset - important when selectively trying to apply transforms
# NOTE: i think the type should be IMAGE and not INTENSITY

if TEST:
    for images in test_subject.values():
        print(images.type)
    
    test_subject.img.data[:3,...].shape
    
    flatten_image_no_transform = torch.flatten(xx.img.data).numpy()
    #plt.hist(flatten_image_no_transform, bins=20)

### TIO API

#### Transforms

* I'm unsure about how useful affine is for kidney slices. On one hand you will never get a kidney slice which on its own is transformed this way, however this may help look at different angles of glomeruli (which themselves at random angles depending on how the kidney was sliced)  

* There are a whole list of transforms that help with (artifacts like spikes, ghosting, motion blurring). However knowing how these samples are prepared and imaged, its unlikely that these effects will be present.

In [26]:
train_transforms = tio.Compose([
    custom_reshape,
    #custom_shrink,
    #reshuffle,
    #tio.Resample(4),
    #tio.ZNormalization(exclude="mask"),
    tio.RandomFlip(),
    tio.RandomAffine(),
    #tio.RescaleIntensity((0,1)),
    custom_normalization,
])

valid_transforms = tio.Compose([
    
    custom_reshape,
    #reshuffle,
    #tio.Resample(4),
    #tio.ZNormalization(exclude="mask"),
    #tio.RescaleIntensity((0,1)),
    custom_normalization,
])

train_dataset = tio.SubjectsDataset(train_subjects, transform=train_transforms)
valid_dataset = tio.SubjectsDataset(valid_subjects, transform=valid_transforms)

label_samples = {
    0: 8,
    1: 3,
    2: 2
}

# patch should be size that is continuously divisible by 2 
# ie. not 100, as 100>50>25>12.5 

patch_size = (256, 256, 1) 

sampler = tio.data.LabelSampler(patch_size, label_probabilities=label_samples)

queue_length = 100
samples_per_volume = 100

train_queue = tio.Queue(
    train_dataset,
    queue_length,
    samples_per_volume,
    sampler,
    num_workers=0,
    #shuffle_subjects=False,
    #shuffle_patches=False
)

valid_queue = tio.Queue(
    valid_dataset,
    queue_length,
    samples_per_volume,
    sampler,
    num_workers=0,
    #shuffle_subjects=False,
    #shuffle_patches=False
)

#train_loader = DataLoader(train_queue, batch_size=16, pin_memory=True)
train_loader = DataLoader(train_queue, batch_size=16)
#valid_loader = DataLoader(valid_queue, batch_size=16, pin_memory=True)
valid_loader = DataLoader(valid_queue, batch_size=16)

In [27]:
label_samples

{0: 8, 1: 3, 2: 2}

### Manual Training

In [28]:
if TEST:    
    one_batch = next(iter(train_loader))

    batch_imgs = one_batch['img'][tio.DATA][:,:3,...].squeeze()
    batch_label = to_3chan(one_batch['mask'][tio.DATA]).squeeze()

    batch_imgs[0].shape

    batch_imgs = one_batch['img'][tio.DATA][:,:3,...].squeeze()
    batch_label = to_3chan(one_batch['mask'][tio.DATA]).squeeze()
    slices = torch.cat((batch_imgs, batch_label))
    image_path = 'batch_patches.png'
    torchvision.utils.save_image(
        slices,
        image_path,
        nrow=8//2,
        normalize=True,
        scale_each=True,
    )

    one_batch['img'][tio.DATA]

### Model Build

UNET Segmentation is the leading NN architecture for medical image analysis. 

The basic UNET architecure is below

<img src = "https://github.com/fastai/fastbook/raw/fb570779062177662fbfde0f5dbb1e9f08dabbee/images/att_00052.png" width=700 />

There are 3 phases:
* **Down/contract**: image size is reduced (typically with a stride 2 configuration) with the focus of feature extraction, which comes from the number of filters. In the example above, a 3-channel input is passed through one convolution with 64 filters. The output of this convolution is then passed through batchnorm and finally a relu. 
    * Note the order of this is (used to be?) up for debate - do you batchnorm *then* activation function, or vice versa?
    * After the second convolution, the model then performs a **maxpool** - you can tell it is the red arrow because the output is half the size of the input (standard maxpool behaviour)
    
Key term:
* **Maxpooling**: feature map outputs of convolutions record the *precise position* of features. However small movements in the positon of the feature (ie. cropping, rotation, shifting) result in a different feature map (ie. feature maps are **sensitive to location**). Maxpooling is a **downsampling** technique (ie. will always make the image **smaller** - see down smapling definition below) that takes the max activation value within a maxpool grid (often 2x2 with stride of 2 - hence half the input size). This makes the model **more robust to changes in feature position** in the image.
    * The pooling layer **operates upon each feature map separately** to create **a new set** of the same number of pooled feature maps.


* **Down sampling**: where a lower resolution version of an input signal is created that still contains the large or important structural elements, without the fine detail that may not be as useful to the task

#### Down block

In [29]:
class CustomUnet(nn.Module):
    """
    The user specifies what the first input channels size will be and the ultimate output size will be
    The downblock and upblock functions also take input and out values - but these are PER CONVOLUTION
    They do not necessarily inherit the values specified by the user
    """
    
    def __init__(self, channel_in, channel_out, stride=1, ks=3):
        super(CustomUnet, self).__init__()
        self.down_conv1 = self._downblock(channel_in, 64, stride=stride, ks=ks)
    
    # downward (contracting) block
    def _downblock(self, n_in, n_out, stride, ks):
        down_conv = nn.Sequential(
            nn.Conv2d(n_in, n_out, stride=stride, kernel_size=ks, padding=ks//2), # 256/2 = 128
            nn.BatchNorm2d(n_out),
            nn.ReLU(),
            nn.Conv2d(n_out, n_out, stride=stride, kernel_size=ks, padding=ks//2), # 128/2 = 64
            nn.BatchNorm2d(n_out),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=ks, stride=2, padding=ks//2) # 64/2 = 32
        )
        return down_conv
    
    def forward(self, x):
        return self.down_conv1(x)
        


As specified in the comments in the class, there are 3 down sampling steps that take place. Therefore if this were our only step, we would expect the same batch size, but 64 channels (because we specified this an the output), where each filter is 32x32

In [30]:
#custom_unet = CustomUnet(3, 10)

In [31]:
#output_dims = custom_unet(dls.one_batch()[0]).shape

#### Up block

This section is similar to the down block with two notable additional steps:
* Upsampling using `nn.ConvTranspose2d`
* Consuming the output from the oppositse side of the "U"

Also important to note: the inputs to the second and final upblocks **are doubled** since we are stacking two tensors (one from the opposite downblock, one from the previous block)

In [32]:
class CustomUnet(nn.Module):
    """
    The user specifies what the first input channels size will be and the ultimate output size will be
    The downblock and upblock functions also take input and out values - but these are PER CONVOLUTION
    They do not necessarily inherit the values specified by the user
    
    The architecture is 3 down blocks, followed by 3 up blocks
    Output is squeezed if the channel_out is 1 - masks are single channels so this matches the dimensions.
    """
    
    def __init__(self, channel_in, channel_out, stride=1, ks=3):
        super(CustomUnet, self).__init__()
        self.down_conv1 = self._downblock(channel_in, 16, stride=stride, ks=ks)
        self.down_conv2 = self._downblock(16, 32, stride=stride, ks=ks)
        self.down_conv3 = self._downblock(32, 64, stride=stride, ks=ks)
        self.up_conv3 = self._upblock(64, 32, stride=stride, ks=ks)
        self.up_conv2 = self._upblock(32*2, 16, stride=stride, ks=ks) # key to notice the doubling of input size
        self.up_conv1 = self._upblock(16*2, channel_out, stride=stride, ks=ks)
    
    # downward (contracting) block
    def _downblock(self, n_in, n_out, stride, ks):
        down_conv = nn.Sequential(
            nn.Conv2d(n_in, n_out, stride=stride, kernel_size=ks, padding=ks//2), 
            nn.BatchNorm2d(n_out),
            nn.ReLU(),
            nn.Conv2d(n_out, n_out, stride=stride, kernel_size=ks, padding=ks//2), 
            nn.BatchNorm2d(n_out),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=ks, stride=2, padding=ks//2) # 256/2 = 128
        )
        return down_conv
    
    def _upblock(self, n_in, n_out, stride, ks):
        up_conv = nn.Sequential(
            nn.Conv2d(n_in, n_out, stride=stride, kernel_size=ks, padding=ks//2),
            nn.BatchNorm2d(n_out),
            nn.ReLU(),
            nn.Conv2d(n_out, n_out, stride=stride, kernel_size=ks, padding=ks//2), 
            nn.BatchNorm2d(n_out),
            nn.ReLU(),
            nn.ConvTranspose2d(n_out, n_out, stride=2, kernel_size=ks, padding=ks//2, output_padding=ks//2),
        )
        return up_conv
    
    def forward(self, x):
        down_conv1 = self.down_conv1(x)
        down_conv2 = self.down_conv2(down_conv1)
        down_conv3 = self.down_conv3(down_conv2)
        
        up_conv3 = self.up_conv3(down_conv3)
        
        up_conv2 = self.up_conv2(torch.cat([up_conv3, down_conv2], 1))
        up_conv1 = self.up_conv1(torch.cat([up_conv2, down_conv1], 1))
        
        return nn.Sigmoid()(up_conv1)

In [33]:
custom_unet = CustomUnet(3, 1)

#### Loss function

The loss function for this competition is **Dice Loss** - but we need to dive into the details of image segmentation first.

A [blog post by Jeremy Jordan](https://www.jeremyjordan.me/semantic-segmentation/) has been extremely helpful in walking through the basics of image segmentation.

The goal of image segmentation is to **label each pixel** with a **class** of what is being represented. This is often referred to as **dense prediction**

<img src="https://www.jeremyjordan.me/content/images/2018/05/Screen-Shot-2018-05-17-at-9.02.15-PM.png" width=500 />
<img src="https://www.jeremyjordan.me/content/images/2018/05/Screen-Shot-2018-05-16-at-9.36.00-PM.png" width=500 />

##### Why not cross entropy?

The statistical distributions of labels affect training accuracy. Although weighted cross entropy can partially solve this proble, another limitation arises: each pixel is evaluated discretely/independently whether it belongs to a class - it does not consider adjacent pixels. Therefore the outcome is per-pixel loss, as opposed to loss relative to the entire segmented object or boundary.

##### Sigmoid - a refresher

The purpose of Sigmoid is to squeeze values (predictions) to between 0 and 1. This is useful for **binary** classification only, as the model *learns to push one class towards 0 and the second class towards 1*.

Let's get a single batch and run it through our model to see its predictions - the first value in the tuple (`y_preds`). Let's also get the second value in the tuple - the actuals (`y_true`)

In [34]:
"""def dice_loss(y_true, y_pred):
    numerator = 2. * torch.sum(y_pred * y_true)
    denominator = torch.sum(y_pred + y_true)
    return 1 - numerator/denominator"""

class DiceLoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(DiceLoss, self).__init__()

    def forward(self, inputs, targets, smooth=1):
        
        #comment out if your model contains a sigmoid or equivalent activation layer
        #inputs = F.sigmoid(inputs)       
        
        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)
        
        intersection = (inputs * targets).sum()                            
        dice = (2.*intersection + smooth)/(inputs.sum() + targets.sum() + smooth)  
        
        return 1 - dice
    
dice_loss = DiceLoss()    
    
class DiceBCELoss(nn.Module):
    # Formula Given above.
    def __init__(self, weight=None, size_average=True):
        super(DiceBCELoss, self).__init__()

    def forward(self, inputs, targets, smooth=1):
        
        #comment out if your model contains a sigmoid or equivalent activation layer
        #inputs = F.sigmoid(inputs)       
        
        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)
        
        intersection = (inputs * targets).sum()                            
        dice_loss = 1 - (2.*intersection + smooth)/(inputs.sum() + targets.sum() + smooth)  
        BCE = F.binary_cross_entropy(inputs, targets, reduction='mean')
        Dice_BCE = BCE + dice_loss
        
        return Dice_BCE
    
bce_dice_loss = DiceBCELoss()

In [35]:
if TEST:
    custom_unet.to("cpu")
    
    xx = test_subject['mask'][tio.DATA].squeeze()
    yy = torch.where(xx != torch.tensor(1), torch.tensor(0), torch.tensor(1))
    
    y_preds = custom_unet(one_batch['img'][tio.DATA].squeeze().cpu())
    y_true = torch.where(one_batch['mask'][tio.DATA].squeeze() != torch.tensor(1), torch.tensor(0), torch.tensor(1))
    
    y_true.to("cpu")
    
    #assert y_true[0].shape == y_preds[0].shape
    
    print(dice_loss(y_preds, y_true.float()))
    
    print(acc_metric(y_preds, y_true))
    #y_preds.shape

In [36]:
def acc_metric(predb, yb):
    return (torch.round(predb) == yb).float().mean()

Cannot use fastai - need to figure out why

In [37]:
import time
epochs = 1

def trainer(dl_train, dl_valid, model, loss_fxn, optimizer, metric_fxn, epochs=1):
    start = time.time()
    
    # set model to cuda - look into the .device() method
    model.cuda()
    
    # collect training stats
    train_loss, valid_loss = [], []
    
    # step for printing loss and metric
    step = 0
    
    for epoch in range(epochs):
        
        for stage in ["train", "valid"]:
            if stage == "train":
                model.train(True)
                dataloader = dl_train
            else:
                model.train(False)
                dataloader = dl_valid

            running_loss = 0.0
            running_metric = 0.0
        
            for i, data in enumerate(dataloader):
                # squeeze the depth dimension as our model doesn't account for this but TorchIO utilises this
                inputs = data['img'][tio.DATA].squeeze()
                labels = data['mask'][tio.DATA].squeeze()
                # need to override the labels to ensure we are only guessing presence of glomeruli
                # remember the additional category was introduced just to improve sampling selection area
                labels = torch.where(labels != torch.tensor(1), torch.tensor(0), torch.tensor(1))
                
                inputs = inputs.cuda()
                labels = labels.float().cuda()
                
                if stage == "train":
                    
                    # set the grads to zero
                    optimizer.zero_grad()

                    # forward
                    outputs = model(inputs)

                    # backward
                    loss = loss_fxn(outputs, labels)
                    loss.backward()
                    optimizer.step()
                    
                else:
                    with torch.no_grad():
                        outputs = model(inputs)
                        loss = loss_fxn(outputs, labels)
                        
                metric = metric_fxn(outputs, labels)
                
                del inputs, labels, outputs
                gc.collect()
                
                running_loss += loss*dataloader.batch_size
                running_metric += metric*dataloader.batch_size
                
                if step % 10 == 0:
                    print('Current step: {}  Loss: {}  Metric: {}  AllocMem (Mb): {}'.format(step, loss, metric, torch.cuda.memory_allocated()/1024/1024))
                
            epoch_loss = running_loss/len(dataloader.dataset)
            epoch_metric = running_metric/len(dataloader.dataset)
            
            train_loss.append(epoch_loss) if stage=='train' else valid_loss.append(epoch_loss)
            
    time_elapsed = time.time() - start
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))    
    
    return train_loss, valid_loss

In [38]:
opt = torch.optim.Adam(custom_unet.parameters(), lr=0.001)
#train_loss, valid_loss = trainer(train_loader, valid_loader, custom_unet, bce_dice_loss, opt, acc_metric)

It is **important to summarise** what is going on here. There are 5 keys steps:

0. Extract `x` and `y` from batch (and perform any updates to them, ie. modify their shape)
1. **forward**: calculate the **logits** by passing the input through the model
2. Calculate the **loss function**
3. **zero the gradients** - this is zeroing any grads from previous loops, as otherwise these would *aggregate*
    * Note in the loop above, we are calling `opt.zero_grad()` - this is because the parameters also exist in the optimizer. However we could have also written `model.zero_grad()` or `model.parameters.grad.zero_()` (notice the trailing `_` which means update in place)
4. **backward**: calculates the partial derivatives of the loss function with respect to the parameters
    * More technically, the underlying function is `params.grad.add_(dloss/dparams)`
5. **step**: update the parameters using the gradients
    * Again this is done on the `optimizer.step()` because like before, the parameters exist in the optimizer and the Pytorch package knows how to update these
    * Critical to remember this is equivalent to: **`with torch.no_grad(): params -= params.grad * lr`**, which you may also add batchnorm here as well

One final summary: gradients are calculated using the loss (the partial derivative of loss with respect to params) by calling `loss.backward()` on the **loss function**, but those gradients are accessible on the **parameters** `params.grad`. These gradients are then multiplied by the elarning rate and subtracted from the previous params

calculate the accuracy of guessing all 0's and the proportion of 1s to 0s in each image

also try a new Unet, different tile size, different sampling proportions, different affine matrix, and more transforms, and the normal loss function

What about Unet transfer learning or transformer?

In [39]:
#test_ds = train_dataset[0]

In [40]:
#test_ds_item_counts = torch.unique(test_ds.mask.data, return_counts=True)

In [41]:
#(test_ds_item_counts[1][1] / test_ds_item_counts[1][2])*100

## Pytorch Lightning

In [42]:
#sample_loader = next(iter(train_loader))

In [43]:
class OLDLitUNETDataLoader(pl.LightningDataModule):
    def __init__(self, train_loader, valid_loader):
        super().__init__()
        self.train_loader = train_loader
        self.valid_loader = valid_loader
        
    def train_dataloader(self):
        return self.train_loader
    
    def val_dataloader(self):
        return self.valid_loader
    
    def transfer_batch_to_device(self, batch, device):
        # squeeze the depth dimension as our model doesn't account for this but TorchIO utilises this
        x = batch['img'][tio.DATA].squeeze()
        y = batch['mask'][tio.DATA].squeeze()
        # need to override the labels to ensure we are only guessing presence of glomeruli
        # remember the additional category was introduced just to improve sampling selection area
        y = torch.where(y != torch.tensor(1), torch.tensor(0), torch.tensor(1))
        return x.to(device), y.to(device)
    
    

class LitUNET(pl.LightningModule):
    def __init__(self, train_dl, valid_dl, model, loss_fxn, bs=32, learning_rate=3e-3, manual_optimization=False):
        super().__init__()
        #self.save_hyperparameters()
        self.train_dl = train_dl
        self.valid_dl = valid_dl
        self.model = model
        self.loss_fxn = loss_fxn
        self.lr = learning_rate
        self._manual_optimization = manual_optimization
        if self._manual_optimization:
            self.training_step = self.training_step_manual
            
    def forward(self, x):
        out = self.model(x)
        return out
    
    def train_dataloader(self):
        return self.train_dl
    
    def val_dataloader(self):
        return self.valid_dl

    def training_step(self, batch, batch_idx):
        # squeeze the depth dimension as our model doesn't account for this but TorchIO utilises this
        x = batch['img'][tio.DATA].squeeze()
        y = batch['mask'][tio.DATA].squeeze()
        # need to override the labels to ensure we are only guessing presence of glomeruli
        # remember the additional category was introduced just to improve sampling selection area
        y = torch.where(y != torch.tensor(1), torch.tensor(0), torch.tensor(1))
        y_hat = self.model(x)
        loss = self.loss_fxn(y_hat, y)
        self.log('train_loss', loss, on_epoch=True)
        return loss

    def validation_step(self, batch, batch_idx):
        # squeeze the depth dimension as our model doesn't account for this but TorchIO utilises this
        x = batch['img'][tio.DATA].squeeze()
        y = batch['mask'][tio.DATA].squeeze()
        # need to override the labels to ensure we are only guessing presence of glomeruli
        # remember the additional category was introduced just to improve sampling selection area
        y = torch.where(y != torch.tensor(1), torch.tensor(0), torch.tensor(1))
        y_hat = self.model(x)
        loss = self.loss_fxn(y_hat, y)
        self.log('valid_loss', loss, on_step=True)

    def test_step(self, batch, batch_idx):
        # squeeze the depth dimension as our model doesn't account for this but TorchIO utilises this
        x = batch['img'][tio.DATA].squeeze()
        y = batch['mask'][tio.DATA].squeeze()
        # need to override the labels to ensure we are only guessing presence of glomeruli
        # remember the additional category was introduced just to improve sampling selection area
        y = torch.where(y != torch.tensor(1), torch.tensor(0), torch.tensor(1))
        y_hat = self.model(x)
        loss = self.loss_fxn(y_hat, y)
        self.log('test_loss', loss)

    def configure_optimizers(self):
        # self.hparams available because we called self.save_hyperparameters()
        return torch.optim.Adam(self.parameters(), lr=self.lr)



In [44]:
class LitUNETDataLoader(pl.LightningDataModule):
    def __init__(self, train_loader, valid_loader):
        super().__init__()
        self.train_loader = train_loader
        self.valid_loader = valid_loader
        
    def train_dataloader(self):
        return self.train_loader
    
    def val_dataloader(self):
        return self.valid_loader
    
    def transfer_batch_to_device(self, batch, device):
        # squeeze the depth dimension as our model doesn't account for this but TorchIO utilises this
        x = batch['img'][tio.DATA].squeeze()
        y = batch['mask'][tio.DATA].squeeze()
        # need to override the labels to ensure we are only guessing presence of glomeruli
        # remember the additional category was introduced just to improve sampling selection area
        y = torch.where(y != torch.tensor(1), torch.tensor(0), torch.tensor(1))
        return x.to(device), y.to(device)
    
    

class LitUNET(pl.LightningModule):
    def __init__(self, model, loss_fxn, bs=32, learning_rate=3e-3, num_workers=0, manual_optimization=False):
        super().__init__()
        #self.save_hyperparameters()
        self.model = model
        self.loss_fxn = loss_fxn
        self.lr = learning_rate
        self._manual_optimization = manual_optimization
        if self._manual_optimization:
            self.training_step = self.training_step_manual
            
    def forward(self, x):
        out = self.model(x)
        return out

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.model(x)
        loss = self.loss_fxn(y_hat, y)
        self.log('train_loss', loss, on_epoch=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.model(x)
        loss = self.loss_fxn(y_hat, y)
        self.log('valid_loss', loss, on_step=True)

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.model(x)
        loss = self.loss_fxn(y_hat, y)
        self.log('test_loss', loss)

    def configure_optimizers(self):
        # self.hparams available because we called self.save_hyperparameters()
        return torch.optim.Adam(self.parameters(), lr=self.lr)



In [45]:
test_list = list(range(1,10))

In [48]:
test_list[round(len(test_list)*(1-0.5)):]

[5, 6, 7, 8, 9]

In [51]:
class LossSpikeAnalyzer(Callback):
    def __init__(self):
        self.train_batch_loss = []
        self.valid_batch_loss = []
    def on_train_batch_end(self, trainer, pl_module, outputs, batch, batch_idx, dataloader_idx):
        callback_stats = trainer.callback_metrics
        if "train_loss_step" in callback_stats.keys():
            self.train_batch_loss.append(callback_stats["train_loss_step"].item())
            #if len(self.train_batch_loss) > 10:
                #print(f"The rolling loss is {np.asarray(self.train_batch_loss).mean()}")
                #print(f"The mean of the last 50% of batches is {np.asarray(self.train_batch_loss[round(len(self.train_batch_loss)*(1-0.5)):]).mean()}")
            #print(f"The shape of the batch is {batch['img'][tio.DATA].shape} - the batch actually returns {batch.keys()} - and {outputs}, and the loss is {np.asarray(self.batch_loss).mean()}")
            if callback_stats["train_loss_step"].item() > 0.9 or batch_idx % 60 == 0:
                img_list = []
                with torch.no_grad():
                    pl_module.eval()
                    preds = pl_module(batch['img'][tio.DATA].squeeze().cuda())
                    pl_module.train()
                for i, img in enumerate(batch['img'][tio.DATA].squeeze()):
                    mask = torch.where(batch['mask'][tio.DATA].squeeze()[i,...].unsqueeze(0) != torch.tensor(1), torch.tensor(0), torch.tensor(1))
                    mask = to_3chan(mask, 0)
                    pred = to_3chan(preds[i,...],0)
                    img_list.append(img)
                    img_list.append(mask)
                    img_list.append(pred.cpu())
                grid = torchvision.utils.make_grid(
                    img_list,
                    nrow=3,
                )
                self._save(grid, batch_idx, callback_stats["train_loss_step"].item())
    
    def on_validation_batch_end(self, trainer, pl_module, outputs, batch, batch_idx, dataloader_idx):
        print(f"The VALID BATCH END metrics are {trainer.callback_metrics}")
        
    def on_validation_end(self, trainer, pl_module):
        print(f"Note that the metrics when VALIDATION ENDS are {trainer.callback_metrics}")

    @staticmethod
    def _show(img):
        npimg = img.numpy()
        plt.imshow(np.transpose(npimg, (1,2,0)), interpolation='nearest')
        
    @staticmethod
    def _save(img, batch_idx, loss):
        npimg = img.numpy()
        loss = str(round(loss,2)).replace(".", "_")
        plt.imsave(path/"training_image_logs"/f"{batch_idx}__{loss}.png", np.transpose(npimg, (1,2,0)))

In [52]:
data = LitUNETDataLoader(train_loader, valid_loader)
model = LitUNET(custom_unet, dice_loss, num_workers=16)
trainer = pl.Trainer(gpus=1, callbacks=[LossSpikeAnalyzer()])
trainer.fit(model, data)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name     | Type       | Params
----------------------------------------
0 | model    | CustomUnet | 123 K 
1 | loss_fxn | DiceLoss   | 0     
----------------------------------------
123 K     Trainable params
0         Non-trainable params
123 K     Total params


Validation sanity check:  50%|█████████████████████████████▌                             | 1/2 [00:06<00:06,  6.40s/it]The VALID BATCH END metrics are {}
Note that the metrics when VALIDATION ENDS are {'valid_loss_epoch': tensor(0.7063, device='cuda:0'), 'valid_loss': tensor(0.7316, device='cuda:0')}
Epoch 0:  50%|█████████████████████████                         | 25/50 [01:22<01:22,  3.31s/it, loss=0.783, v_num=122]
Validating: 0it [00:00, ?it/s]
Validating:   0%|                                                                               | 0/25 [00:00<?, ?it/s]The VALID BATCH END metrics are {'valid_loss_epoch': tensor(0.7063, device='cuda:0'), 'valid_loss': tensor(0.7316, device='cuda:0'), 'train_loss_step': tensor(0.8007, device='cuda:0')}
The VALID BATCH END metrics are {'valid_loss_epoch': tensor(0.7063, device='cuda:0'), 'valid_loss': tensor(0.7316, device='cuda:0'), 'train_loss_step': tensor(0.8007, device='cuda:0')}
Epoch 0:  56%|████████████████████████████                

C:\Users\Craig\miniconda3\envs\kidneys-cv\lib\site-packages\pytorch_lightning\utilities\distributed.py:50: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)


1

In [49]:
#[img.unlink() for img in (path/"training_image_logs").glob("*") if img.name != ".ipynb_checkpoints"]

## Merge with fastai

In [50]:
dls = DataLoaders(train_loader, valid_loader)
#dls = DataLoaders.from_dsets(train_queue, valid_queue)

NameError: name 'DataLoaders' is not defined

So now we know that we have predictions and targets with the same shape, and ultimately we want the predictions to be as close to 0 or 1 for each pixel.

The **dice loss** function...

In [ ]:
model = custom_unet
model.cuda()
learn = Learner(train_dl, model, loss_func=bce_dice_loss, metrics=acc_metric)

In [ ]:
#learn.fit_one_cycle(1)

In [ ]:
??Learner

## Create background segmentation

### TODO: 
- [] incorporate scale into below functions
- [] convert to a class as a list of functions is confusing

In [ ]:
#encs = get_single_encs(get_id_by_index(0))
#baseimage = get_single_img(get_id_by_index(0))

#show_single_img_and_mask(get_id_by_index(1))

### Tile images

I was concerned about memory management - note on this below

In [ ]:
TILE_SIZE = 256
s_th = 40  #saturation blancking threshold
p_th = 200*TILE_SIZE//TILE_SIZE #threshold for the minimum number of pixels
reduce = 1
x_tot,x2_tot = [],[]

def image_tiles(df: pd.DataFrame = train_df):
    
    (path/"formatted"/str(TILE_SIZE)/"imgs").mkdir(parents=True, exist_ok=True)
    impath = path/"formatted"/str(TILE_SIZE)/"imgs"
    (path/"formatted"/str(TILE_SIZE)/"masks").mkdir(parents=True, exist_ok=True)
    maskpath = path/"formatted"/str(TILE_SIZE)/"masks"
    
    output = []
    
    for index, item in tqdm(enumerate(df.id), total=len(df)):

        # read images and generate masks
        img_id = get_id_by_index(index)
        img = get_single_img(img_id)
        mask = get_mask(img_id)

        # add padding to make the image dividable into tiles
        shape = img.shape
        pad0 = (reduce*TILE_SIZE - shape[0]%(reduce*TILE_SIZE))%(reduce*TILE_SIZE)
        pad1 = (reduce*TILE_SIZE - shape[1]%(reduce*TILE_SIZE))%(reduce*TILE_SIZE)
        img = np.pad(img,[[pad0//2,pad0-pad0//2],[pad1//2,pad1-pad1//2],[0,0]],
                    constant_values=0)
        mask = np.pad(mask,[[pad0//2,pad0-pad0//2],[pad1//2,pad1-pad1//2]],
                    constant_values=0)

        #split image and mask into tiles using the reshape+transpose trick
        img = cv2.resize(img,(img.shape[1]//reduce,img.shape[0]//reduce),
                         interpolation = cv2.INTER_AREA)
        img = img.reshape(img.shape[0]//TILE_SIZE,TILE_SIZE,img.shape[1]//TILE_SIZE,TILE_SIZE,3)
        img = img.transpose(0,2,1,3,4).reshape(-1,TILE_SIZE,TILE_SIZE,3)

        mask = cv2.resize(mask,(mask.shape[1]//reduce,mask.shape[0]//reduce),
                          interpolation = cv2.INTER_NEAREST)
        mask = mask.reshape(mask.shape[0]//TILE_SIZE,TILE_SIZE,mask.shape[1]//TILE_SIZE,TILE_SIZE)
        mask = mask.transpose(0,2,1,3).reshape(-1,TILE_SIZE,TILE_SIZE)

        #write data
        for i,(im,m) in enumerate(zip(img,mask)):
            #remove black or gray images based on saturation check
            hsv = cv2.cvtColor(im, cv2.COLOR_BGR2HSV)
            h, s, v = cv2.split(hsv)
            if (s>s_th).sum() <= p_th or im.sum() <= p_th: continue

            x_tot.append((im/255.0).reshape(-1,3).mean(0))
            x2_tot.append(((im/255.0)**2).reshape(-1,3).mean(0))

            im = cv2.imencode('.png',cv2.cvtColor(im, cv2.COLOR_RGB2BGR))[1]
            cv2.imwrite((impath/f'{img_id}_{i}.png').as_posix(), im)
            #m = cv2.imencode('.png',m)[1]
            #cv2.imwrite(str(path.absolute()/"train"/"formatted"/"masks"/f'{index}_{i}.png'), m)
            output.append((img_id, i, im, m))
            #if i % 1000 == 0:
                #print(type(im))
                #return plt.imshow(im)
            del i, im, m
        del img, mask, pad0, pad1, item, index
    output_data = pd.DataFrame(output, columns=["parent_img_id", "tile_index", "img", "mask"])
    del output
    gc.collect()
    return output_data
    #print(type(output_data['img'][0]))
    #output_data.to_csv(path/"train"/"train_tiles.csv")

In [ ]:
train_data = image_tiles(train_df)

In [ ]:
# Don't recreate the dataset everytime - pull from local directory if available as pickle file
# I understand pickles aren't safe - so only do this in your local env and never open an unfamiliar pickle file
if not (path/"datastore.pkl").exists():
    train_data = image_tiles(train_df)
    train_data.to_pickle(path/"datastore.pkl")
if (path/"datastore.pkl").exists():
    train_data = pd.read_pickle("datastore.pkl")

#### A note on Python memory management

I decided to take a look at how much RAM this programme has taken thus far - when I opened up Windows Task Manager I noticed that after the above loop ran, my memory usage was at 64% (20.3GB out of 32GB). However when I look at the size of the output DataFrame, it is only 16.8MB: 

In [ ]:
train_data.info(memory_usage="deep")

Having spent several hours researching this, I have come to conclude the following:

* Python memory manages on its own - this is one of its advantages to lower-level languages like C.
    * Any attempt to help with this doesn't really work. I tried using `del` and then `gc.collect()` after several points in the loop above and not only did they not reduce the memory usage at the end of the run, but adding `gc.collect()` everywhere slowed down the loop significantly (like 4x).
    * I think this approach may be useful if you were running out of memory mid-run (ie. getting an `out of memory` error). However this wasn't the case for me
    * I tried memory management libraries like `objgraph` but they didn't really help shed any more light on this
* From some of the reading I've done, it looks like Windows may not "recall" the unused memory from Python after it has been used (it assumes the programme may need similar amounts in the future). Therefore despite my Task Manager Memory still sitting at 64% used, I can only conclude that this is misleading - and Windows would recover this if I ran another programme
    * Again it seems that objects in Python are assigned for deletion (either automatically or when `del` is used) - however all this means is that it *can* be overwritten (ie. when some other programme or Python variable needs it). An object isn't just overwritten and assigned with zeroes or null - it will replace these tagged objects only when required
    * *Update*: a simple test has confirmed this. I opened Microsoft Flight Simulator 2020 (knowing it was memory intensive) and Python's memory usage dropped by 30% instantaneously. Closing MFS (ie. removing off memory) has reduced the current baseline memory usage now to 16GB (50% max capacity). This confirms that Windows 10 looks like Python is storing onto objects in memory, when in fact that memory is available.
    * Finally, while this experiment has ensured my approach is still OK (ie. using DataFrames instead of downloading images locally into files), memory management is still something to be aware of - ensuring objects can be overwritten by other variables/programmes (by not unnecessarily storing large objects in your programme) helps ensure the system stays functional
    

In [ ]:
#image stats
img_avr =  np.array(x_tot).mean(0)
img_std =  np.sqrt(np.array(x2_tot).mean(0) - img_avr**2)
print('mean:',img_avr, ', std:', img_std)

### Get sample images

Note the key here is converting the mask numpy array to HSV values through **changing colorspaces**. The function `cv2.cvtColor` takes an image and a colorspace to change to. In our case, we are converting the numpy arrays (which only contain 0s and 1s - we replaced 0s with 1s in the `rle2mask` function) to HSV values, and summing the number of 1s in each image.

We then show a random tile set depending on if we want the presence of glomeruli (generally yes).

In [ ]:
def to_3_channels(x):
    return np.stack((x, x, x), 2)

# Function to return images with or without glom
def get_glom_idxs(glom: bool = True):
    idxs=[]
    for i, item in enumerate(train_data["mask"]):
        item = to_3_channels(item)
        h,s,v = cv2.split(cv2.cvtColor(item, cv2.COLOR_BGR2HSV))
        if glom:
            if v.sum() > 0:
                idxs.append(i)
        else:
            if v.sum() < 0:
                idxs.append(i)
    return idxs

def show_tile(index: int=None, glom: bool = True):
    plt.figure(figsize=(16, 10))
    
    if not index:
        index = random.choice(get_glom_idxs(glom=glom))
    mask = train_data["mask"][index]
    img = train_data["img"][index]

    plt.subplot(1, 3, 1)
    plt.imshow(img)
    plt.title(f"Image", fontsize=18)
    
    plt.subplot(1, 3, 2)
    plt.imshow(img)
    plt.imshow(mask, cmap="hot", alpha=0.5)
    plt.title(f"Image + mask", fontsize=18)    
    
    plt.subplot(1, 3, 3)
    plt.imshow(mask, cmap="hot")
    plt.title(f"Mask", fontsize=18)    
    
    return plt.show()

show_tile(241)

## Create DataLoaders

Like the Digit Recognizer notebook, the data here is stored in the DataFrame as pixel values instead of as larger images in local directories. Therefore we need to create a custom `Transform` to process this data 

I noticed something while inspecting the data: not only is the dataset imbalanced, but even if there is a presence of a glomeruli, it could be just a few pixels. So the question is - do we include these or not?

Below we are adding to the dataframe a column that evaluates the relative presence of glomeruli in an image. It evaluates how many pixels a glomeruli take up in an image - and if its below a certain threshold (default 5%) then we tag it as such and may remove for training.

In [ ]:
def glom_present(x, thresh=0.05):
    mask = to_3_channels(x)
    h,s,v = cv2.split(cv2.cvtColor(mask, cv2.COLOR_BGR2HSV))
    if v.sum()>0:
        if v.sum()/(TILE_SIZE**2) < thresh:
            return "partial"
        return "yes"
    return "no"

# apply() over dataframe has been very fast for me in the past 
train_data["glom_present"] = train_data["mask"].apply(glom_present)

The distribution of image type categories

In [ ]:
train_data["glom_present"].value_counts().plot(kind="bar", title="Presence of Glomeruli")

In [ ]:
train_data_no_partial = train_data[train_data["glom_present"] != "partial"]

In [ ]:
class KidneyTransform(Transform):
    def encodes(self, df: pd.core.series.Series):
        #return PILImage.create(tensor(df["img"]))
        return tensor(df["img"]).transpose(2,0).float()

In [ ]:
x_tfms = TfmdLists(train_data, KidneyTransform)

In [ ]:
x_tfms[0].shape

In [ ]:
class GlomMaskImg(Transform):
    def encodes(self, df: pd.core.series.Series):
        #return TensorCategory(df[''])
        mask = to_3_channels(df["mask"])
        h,s,v = cv2.split(cv2.cvtColor(mask, cv2.COLOR_BGR2HSV))
        #return v.sum()
        return PILMask.create(df["mask"])

In [ ]:
class HasGlom(Transform):
    def encodes(self, df: pd.core.series.Series):
        if df["glom_present"] == "yes":
            return 1
        return 0

In [ ]:
class GlomMask(Transform):
    def encodes(self, df: pd.core.series.Series):
        return tensor(df["mask"]).float()

In [ ]:
y_tfms = TfmdLists(train_data_no_partial, GlomMask)

In [ ]:
#plt.imshow(y_tfms[241], cmap="hot")
y_tfms[241].shape

#### Find a place for this

* **Input normalisation**: important because some images or features may have *more variability* than others - if you do not control for this, large values will dominate and cascade through the network resulting in the *exploding gradient* problem. I also think the model will not generalise as well and will learn to identify only features with low variance. However by **subtracting the mean and dividing by stdev** you ensure the variability is consistent across all features.
    * This may not matter as much for images (although it is still important), imagine a tabular dataset with values on completely different scales (cost in dollars vs satisfaction 1-5) - by normalising the data you will have better luck finding the optimal parameters for the lowest loss function
    * Actually this does matter for images - doing this trains the model faster and improves the loss function (avoids local iminima)
    * **NOTE**: you should normalise the validation and test sets using the **same mean and stdev** as the training set

### UNET by hand

#### DataLoaders

In [ ]:
class KidneyTransform(Transform):
    def encodes(self, df: pd.core.series.Series):
        #return PILImage.create(tensor(df["img"]))
        return tensor(df["img"]).transpose(2,0).float()

In [ ]:
x_tfms = TfmdLists(train_data, KidneyTransform)

In [ ]:
x_tfms[0].shape

In [ ]:
class GlomMask(Transform):
    """
    Unlike the x values, this should return a scalar not a tensor - therefore no float()
    """
    def encodes(self, df: pd.core.series.Series):
        return tensor(df["mask"]).long()

In [ ]:
y_tfms = TfmdLists(train_data, GlomMask)

In [ ]:
#plt.imshow(y_tfms[241], cmap="hot")
y_tfms[241].shape

In [ ]:
parent_imgs = train_data.parent_img_id.unique().tolist()
assert len(parent_imgs[:-2]) + len(parent_imgs[-2:]) == len(parent_imgs)

In [ ]:
split_idx = len([im for im in train_data["parent_img_id"].tolist() if im in parent_imgs[:-2]])
splits = [list(range(split_idx)), list(range(split_idx,len(train_data)))]

# need to be careful here - may not want all of these transforms applied to validation set in the future
tfms = [[KidneyTransform], [GlomMask]]
ds = Datasets(train_data, tfms, splits=splits)

In [ ]:
# Having trouble with cuda so creating a new dataloaders on cpu

#dls = ds.dataloaders(bs=64, num_workers=0).cuda()
dls = ds.dataloaders(bs=8, num_workers=0)

#### Training by hand

In [ ]:
def train(dls, model, loss_fn, optimizer, acc_fn, epochs):
    model.cuda()
    
    train_loss, valid_loss = [], []
    
    best_acc = 0.0
    
    for epoch in range(epochs):
        print('Epoch {}/{}'.format(epoch, epochs - 1))
        print('-' * 10)
    
        for phase in ['train', 'valid']:
            if phase == 'train':
                model.train(True)  # Set trainind mode = true
                dataloader = dls.train
            else:
                model.train(False)  # Set model to evaluate mode
                dataloader = dls.valid

            running_loss = 0.0
            running_acc = 0.0

            step = 0

            # iterate over data
            for x, y in dataloader:
                x = x.cuda()
                y = y.cuda()
                step += 1

                # forward pass
                if phase == 'train':
                    # zero the gradients
                    optimizer.zero_grad()
                    outputs = model(x)
                    loss = loss_fn(outputs, y)

                    # the backward pass frees the graph memory, so there is no 
                    # need for torch.no_grad in this training pass
                    loss.backward()
                    optimizer.step()
                    # scheduler.step()

                else:
                    with torch.no_grad():
                        outputs = model(x)
                        loss = loss_fn(outputs, y.long())

                # stats - whatever is the phase
                acc = acc_fn(outputs, y)

                running_acc  += acc*dataloader.bs
                running_loss += loss*dataloader.bs 

                if step % 10 == 0:
                    # clear_output(wait=True)
                    print('Current step: {}  Loss: {}  Acc: {}  AllocMem (Mb): {}'.format(step, loss, acc, torch.cuda.memory_allocated()/1024/1024))
                    # print(torch.cuda.memory_summary())

            epoch_loss = running_loss / len(dataloader.dataset)
            epoch_acc = running_acc / len(dataloader.dataset)

            print('{} Loss: {:.4f} Acc: {}'.format(phase, epoch_loss, epoch_acc))

            train_loss.append(epoch_loss) if phase=='train' else valid_loss.append(epoch_loss)

    time_elapsed = time.time() - start
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))    
    
    return train_loss, valid_loss    

opt = torch.optim.Adam(custom_unet.parameters(), lr=0.01)
#train_loss, valid_loss = train(dls, custom_unet, dice_loss, opt, acc_metric, epochs=1)

#### Training with fastai

In [ ]:
"""del custom_unet, dls, learn
torch.cuda.empty_cache()
gc.collect()"""

In [ ]:
#custom_unet.cuda()

In [ ]:
learn = Learner(dls, custom_unet, loss_func=dice_loss, cbs=ActivationStats(with_hist=True), metrics=acc_metric)

In [ ]:
#torch.cuda.empty_cache()

In [ ]:
learn.fit_one_cycle(1)

In [ ]:
#learn.activation_stats.plot_layer_stats(0)

In [ ]:
learn.summary()

In [ ]:
torch.cuda.empty_cache()

#### Additional notes

* The number of blocks and the input/output numbers is arbitrary - you need to test different combinations (always considering the input size of your image - no sense windling it down to a vector of 1 pixel)
* The only thing you need to create in a model architecture to then be used in fastai is CHECK THIS the `forward` method
    * Is this true? See the function we created above and ran through the first training loop - all it was was a sequential class

## CNN Basics

Again this is entirely based on the fantastic [fastbook Chapter 13](https://github.com/fastai/fastbook/blob/master/13_convolutions.ipynb)

This technique is so popular for analysing images because it is great at finding edges and shapes, which help to differentiate a 3 for a 7, or a car vs a human. More specifically to quote [machinelearningmastery.com](https://machinelearningmastery.com/pooling-layers-for-convolutional-neural-networks/#:~:text=Pooling%20layers%20provide%20an%20approach,presence%20of%20a%20feature%20respectively.):

> stacking convolutional layers in deep models allows layers **close to the input to learn low-level features (e.g. lines)** and **layers deeper in the model to learn high-order or more abstract features, like shapes or specific objects**

Some key terms:
* **kernel**: a little matrix (such as 3x3) that looks for specific patterns in an image - the presence of the pattern returns a number greater than 0; the absence, close to 0.
* **convolution**: applies the kernel across the entire image - each 3x3 section of the image is multiplied elementwise by the 3x3 kernel and the values are **summed together**.

<img src="https://raw.githubusercontent.com/fastai/fastbook/3916b71bdf2f9e587ac82f3c2ef4aabd05b8f51c/images/chapter9_conv_basic.png" width=750/>

Note that kernels can take all kinds of "patterns" - and its the different patterns that can distinguish different shapes within an image. 

Therefore in the image above, if we assume that kernel was set to look for rounded lines, we could conclude that there is a rounded line in the image at that location because **the *sum* of the kernel x image is 94.2** which is much greater than 0.

In [ ]:
# in each of the examples below, the 

top_edge = tensor([[-1,-1,-1],
                   [ 0, 0, 0],
                   [ 1, 1, 1]]).float()

left_edge = tensor([[-1,1,0],
                    [-1,1,0],
                    [-1,1,0]]).float()

diag1_edge = tensor([[ 0,-1, 1],
                     [-1, 1, 0],
                     [ 1, 0, 0]]).float()

diag2_edge = tensor([[ 1,-1, 0],
                     [ 0, 1,-1],
                     [ 0, 0, 1]]).float()

edge_kernels = torch.stack([left_edge, top_edge, diag1_edge, diag2_edge]).unsqueeze(1) # required to be a rank-4 tensor
edge_kernels.shape

In order to preform this calculation with the Pytorch tools, we need to undertsand the basic conv function `F.conv2d`

The first two parameters:
* **input**:: input tensor of shape (minibatch, in_channels, iH, iW)
* **weight**:: filters of shape (out_channels, in_channels, kH, kW)

In place of a mini-batch, we will just use a single image for now representing the mini-batch, with a single channel. However it is important to note that Pytorch is able to perform the convolutions on the entire batch **simultaneously**.

In [ ]:
# return a rank-4 tensor analogous to a minibatch
def get_single_channel_img(idx):
    im = tensor(train_data["img"][idx]) # returns rank 3 tensor with channel in rightmost position
    im = im.transpose(2,0) # move channel to leftmost - cannot use view or reshape
    im = im[0] # take only one of the 3 channels (we are removing data here so this is only for example)
    im = im.unsqueeze(0).unsqueeze(0) # add back the channel we just stripped away, plus add another
    return im.float()

test_image = get_single_channel_img(257)
test_image.shape

In [ ]:
show_image(test_image[0]) # take the first item of the minibatch, even though there is only 1 image

Noiw that we have our image (representing a minibatch) and tensor of kernels, we can run a convolution over the image.

In [ ]:
conv_img = F.conv2d(test_image, edge_kernels)
conv_img.shape

The output of `F.conv2d` is a **channel for every kernel**. Because the kernels look for different things (the patterns of the kernels in our `edge_kernels` object each look for specific shapes), each of the channels output something different:

In [ ]:
show_image(conv_img[0][0]), show_image(conv_img[0][1]), show_image(conv_img[0][2]), show_image(conv_img[0][3])

**Padding** is used to ensure that the output of the convolution produces an image of the same size (and ensure the edge of the image can be processed).

<img src="https://github.com/fastai/fastbook/raw/3916b71bdf2f9e587ac82f3c2ef4aabd05b8f51c/images/att_00030.png" width=750/>

It is a rule of thumb that kernels only have **odd dimensions** (presumably because a kernel with even dimensions would have to have different padding on different axes). In order to calculate the padding for a kernel of size `k` you simply get the **floor division** aka **interger division**.

Therefore in the image above, with a kernel of size 3x3, the padding is:

In [ ]:
3//2

**Stride** is the amount of pixels you more the kernel over. By increasing the **stride from 1 to 2**, the output of the convnet is **twice as small**.

### A model by hand

Create a simple refactored convnet step that uses `nn.Conv2d` instead of `F.conv2d` because the former creates the **weight matrix** (ie. the **values in the kernels**) when we instantiate it.

Remember that with CNNs, we use backpropogation to improve the kernel values of the filters - aka the weight matrices - against which gradients are calculated to improve the loss function.

In [ ]:
def conv(ni, nf, ks=3, act=True):
    res = nn.Conv2d(ni, nf, stride=2, kernel_size=ks, padding=ks//2)
    if act: res = nn.Sequential(res, nn.ReLU())
    return res

In the above function, the output is half the size of the input (due to the stride of 2). Again the kernel size is an odd number, and the interger division of this odd number is the padding. Equally important is specifying the number of filters (ie. the number of small matrices that collect an output for a convolution).

In the simple CNN below, we can see that an image input of 28\*28 pixels is **reduced in size from each convolution** because of the stride. However to combat the possible loss in capacity of each layer, we **increase twofold the number of filters in each convolution** - these are the values suppiled below. Like hidden layers in a perceptron, *we can decide how many filters we want*, and each of them will learn to specialise in something.

Also note that the size of the input isn't needed - the filters in each convolution are **applied over every pixel that is supplied** to it. This is why there is a 1 as input in the first conv.

Finally to get the output of a single value, we **repeat the number of convs until the output is reduced** to 1.

In [ ]:
simple_cnn = sequential(
    conv(1 ,4),            #14x14
    conv(4 ,8),            #7x7
    conv(8 ,16),           #4x4
    conv(16,32),           #2x2
    conv(32,2, act=False), #1x1
    Flatten(),
)

For colour images, our kernels grow in the third dimension with the number of channels supplied - ie. the same kernel isn't used for each of the channels. 

**Bias** is present for every kernel - each kernel has a correspondinding bias, again much like the perceptron. 

### CNN Considerations

Consider the *number of output features in the first layer and how it relates to the kernel size*. If we were to increase that number from 4 to 8, we would have a kernel size of 3x3=9 trying to differentiate on 8 features - this is hard for them to learn much of anything. But if we increase the kernel size, it is *more likely that 25 pixels can distinguish 8 unique and useful patterns*

In [ ]:
def simple_cnn():
    return sequential(
        conv(1 ,8, ks=5),        #14x14
        conv(8 ,16),             #7x7
        conv(16,32),             #4x4
        conv(32,64),             #2x2
        conv(64,10, act=False),  #1x1
        Flatten(),
    )

But how do we evaluate model effectiveness? Through **callbacks**

We can use the `ActivationStats` callback to evaluate the following scenarios:

* **Mean** of activations - ideally should be normally distributed and its change should be smooth (consistent) during training
* **Standard deviation** of activations - also its change should be smooth (consistent) during training
* **% of activations near 0** - we want to **reduce this as much as possible** - activations of/at zero don't provide any benefit (0x0=0 and cause the subsequent layers to also have 0s) and **take up valuable space**
    * *Poor initialisation* **compounds the frequency of 0 activations** over the layers and results in most of the final layers being 0:
    ![](https://cdn.craigstanton.com/images/ai/fastai/bad_training_stats.png)

By watching the `ActivationStats` during training by plotting `learn.activation_stats.plot_layer_stats(0)` you can see how these change and make 

Some of the solutions to the issues above could be:
* increasing batch size
* variable learning rate
* batch normalisation

#### 1cycle Training

If you have randomly initialised weights, they clearly are not good. Therefore having a high learning rate at the start of training risks missing the loss minimum and thus wastes time (or completely disguises - causing divergence) finding optimal weights.

> Remember, `weights.data -= weights.grad * learning_rate`. 

Again if the learning rate is high when we are just starting out while using random values, you are moving the weights massively - how can they consistently move in the right direction? Equally we don't want a high learning rate near the end of training. Therefore, we should change the learning rate during training, from low, to high, and then back to low again

fastai scales the learning rate up and down - but you need to specify the max lr to be used. Use the `fit_one_cycle` method you can specify the following parameters:
* *lr_max*: The highest learning rate that will be used (this can also be a list of learning rates for each layer group, or a Python slice object containing the first and last layer group learning rates)
* *div*: How much to divide lr_max by to get the starting learning rate
* *div_final*: How much to divide lr_max by to get the ending learning rate
* *pct_start*: What percentage of the batches to use for the warmup
* *moms*: A tuple (mom1,mom2,mom3) where mom1 is the initial momentum, mom2 is the minimum momentum, and mom3 is the final momentum

Note: **high learning rates aren't all bad**. Higher learning rates in the middle of training allow for **faster training** and also help **prevent overfitting** (by *passing over sharp local minima* - and thus making the model more **generalisable**). By preventing overfitting (passing the sharp local minima), we satisfy the rule that a model that generalizes well is one whose loss would not change very much if you changed the input by a small amount.

#### Batch normalisation

The 1cycle training doesn't completely solve our training inefficiency - there is still a propensity to return the parameters to zero. Using fastai's *colorful dimensions* we can visualise the distribution of parameter values (through colorful histograms) with the goal of having normally distributed activations.

The problem arises that the distibution of each layer *changes during training* as the previous layers impact the next layers - this requires lower learning rates (and thus slower training) and careful parameter initialisation. This phenomenon is called **internal covariate shift**.

**Batch normalisation** (aka **batchnorm**) tries to counter this by using the *mean and standard deviation of the **prior layer** to **normalise the next layer***. This technique allows us to use much higher learning rates and be less careful about initialization.

However there is a catch (of course!) - with normalisation alone, we "dampen" activations that may require a high value (ie. a very specific attribute of the image is a huge indicator of a class - therefore specific activations are required to be really high in order to make accurate predictions). To counter this, we add **two learnable parameters `gamma` and `beta`**. The output of each layer is therefore `gamma*y + beta` where `y` has been normalised with the previous layer's parameter mean and standard deviation.

As stated, models with batchnorm tend to generalise well. To quote the fastbook directly:
> most researchers believe that the reason for this is that batch normalization adds some extra randomness to the training process. Each mini-batch will have a somewhat different mean and standard deviation than other mini-batches. Therefore, the activations will be normalized by different values each time. In order for the model to make accurate predictions, it will have to learn to become robust to these variations. In general, adding additional randomization to the training process often helps.

This is a helpful reminder - training happens over a batch, where all images are processed simultaneously.

In [ ]:
??nn.BatchNorm2d

## Critical concepts

* The size of the input image isnt specified as the conv2d is applied over every pixel that is supplied to it
* In order to end up with an output of a single value, we often repeat the number of convs until this happens (ie. number of convs needed is 28 / 2...)
* The first input of `1` looks like a placeholder. However each of the next conv layers takes the output of the previous one
* Because we are using a stride of 2, the output of each layer is a vector that is half the size of the input. To prevent the lack of feature recognition as this happens, we doube the number of filters in each conv
* Jargon: filters vs channels vs kernels vs weight matrix
    * **kernel**: a symmetrical matrix of weights applied to each pixel in an image
    * **filter**: for a 2D convolution, a filter is the same as a kernel. For a 3D convolution, the filter refers to the collection of kernels as a unit
        * Also important that filters are often discussed in terms of number of filters (ie. the second layer has 30 filters)
    * **channels**: also mean filters, but can also refer to the input image - channels in this context are different
    * **weight matrix**: the matrix of values (ie. the kernel values) that are updated on the backwards pass
    * **activations**: the actual values of the weight matrix/kernels
   

A final note on CNNs: 
> Although the universal approximation theorem shows that it should be possible to represent anything in a fully connected network in one hidden layer, we've seen now that in practice we can train much better models by being thoughtful about network architecture.